In [1]:
import numpy as np
import matplotlib.pyplot as plt

import src.utils as utils
import src.rna_utils as rna_utils

In [2]:
q = 4
L = 8

positions = rna_utils.RNA_POSITIONS

In [3]:
# get Fourier coefficients

Calculating free energies...


100%|██████████| 65536/65536 [02:58<00:00, 367.90it/s]


FileNotFoundError: [Errno 2] No such file or directory: '../results/rna_data.npy'

In [ ]:
emp_beta = rna_utils.calculate_rna_fourier_coefficients(save=True)

In [ ]:
# Calculate fraction of variance explained by empirical coefficients
bm_fv = utils.calc_frac_var_explained(emp_beta)

In [ ]:
# get GWHT coefficients
emp_beta_gwht = rna_utils.calculate_rna_gwht_coefficients(save=True)

In [ ]:
# Calculate fraction of variance explained by empirical coefficients
bm_fv_gwht = utils.calc_frac_var_explained(emp_beta_gwht)

In [ ]:
plt.plot(bm_fv[:300], label = "Fourier")
plt.plot(bm_fv_gwht[:300], label = "GWHT")
plt.legend()
plt.show()

In [ ]:
beta_tensor = np.abs(np.reshape(emp_beta_gwht, [q] * L)) ** 2
diff_blocks = []
for _ in range(50):
    ind = np.unravel_index(beta_tensor.argmax(), beta_tensor.shape)
    # print(beta_tensor[ind], ind)
    beta_tensor[ind] = 0
    block_ind = np.array(ind)
    block_ind[block_ind != 0] = 1
    diff_blocks.append(block_ind)

diff_blocks = np.unique(np.array(diff_blocks), axis = 0)
print(diff_blocks.shape)

print(np.unique(np.sum(diff_blocks, axis = 1), return_counts=True))

In [ ]:
beta_tensor = np.abs(np.reshape(emp_beta_gwht, [q] * L)) ** 2
diff_blocks = []
energy = np.zeros((L, L, 9))

for i in range(L):
    for j in range(i+1, L):
        ind = [0] * L
        block_values = []
        for x1 in range(1, 4):
            for x2 in range(1, 4):
                ind[i] = x1
                ind[j] = x2
                block_values.append(beta_tensor[tuple(ind)])
        print(np.mean(block_values), block_values)
        energy[i, j, :] = np.array(block_values).flatten()

In [ ]:
flat_energy = energy.flatten()
for i in range(L ** 2):
    plt.scatter(np.arange(i * 9, i * 9 + 9), np.log10(flat_energy[i * 9 : i * 9 + 9]))
plt.show()

In [ ]:
beta_power = np.abs(emp_beta) ** 2
for _ in range(20):
    ind = beta_power.argmax()
    print(beta_power[ind], ind)
    beta_power[ind] = 0
